In [1]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

from keras import backend as K
from keras.models import load_model

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2
from statistics import mean 

import time
import pickle
import math  

# for scaling and inverse_transform
from sklearn.preprocessing import MinMaxScaler

from jaad_data import JAAD

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

In [3]:
def bb_distance(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = (boxA[0] + boxA[2]) /2
	yA = (boxA[1] + boxA[3]) /2
	xB = (boxB[0] + boxB[2]) /2
	yB = (boxB[1] + boxB[3]) /2
	print(xA,yA,xB,yB) 
 
	# compute the dist
	dist = math.sqrt((xA - xB) * (xA - xB) + (yA - yB)*(yA - yB))
	#print(dist)
 
	return dist

In [4]:
#model_path = "30f_conf4_future-model.h5"
#model_path = "conf5_30ffuture-model.h5"
#model_path = "conf6_1000e_30ffuture-model.h5"
#model_path = "conf7_1000e_15ffuture-model.h5"
#model_path = "conf8_300e_30ffuture-model.h5"
model_path = "conf9_300e_60ffuture-model.h5"

model = load_model(model_path)
n_seq = 15
n_seq_future = 60

In [5]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')
imdb = JAAD(data_path = jaad_path)

In [6]:
videos = ["video_0%s"%str(i) for i in range(317,347)]
#videos = ["video_00%s"%str(i) for i in range(71,100)]

In [7]:
len(videos)

30

In [13]:
avg_prediction_speed = list()
accuracy = list()
print(len(avg_prediction_speed), len(accuracy))
for v in videos:
    anno = imdb._get_annotations(v)

    bb_cross={}
    ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

    for i, p in enumerate(ped_b):
        #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
            bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
            #print(p)
    print(v)
    print(ped_b)
    
    for p in ped_b:
      #  print(v, p)
        lstm_seq = ( bb_cross[p])
        #print(type(lstm_seq))
        frames = (anno['ped_annotations'][p]['frames'][0],
        len(anno['ped_annotations'][p]['frames']))

        df = pd.DataFrame(lstm_seq)
        df = df.astype('float32')

        #print(type(scaler))
        #print(scaler.data_max_)
        #print(scaler.data_min_)
        scaler = MinMaxScaler(feature_range=(0, 1))
        lstm_seq = scaler.fit_transform(lstm_seq)
        pred = True
        i = 0
        total_time = 0.0
        total_positive = 0

        while( i < lstm_seq.shape[0] - (n_seq + n_seq_future) -1):
            batch = lstm_seq[i:i + n_seq] # n_seq number of element with last index (n_seq-1)

            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            #print(y_hat)
            #print(len(batch))

            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            #print(inv_yhat)

            # future bounding box from gt

            boxA =  df[i+(n_seq + n_seq_future) - 1 :i + (n_seq + n_seq_future)].values[0]
            # future bounding box from prediction
            boxB = inv_yhat

            #print(boxA)
            #print(boxB)

            score = bb_intersection_over_union(boxA, boxB)
            if score > 0.5:
                total_positive += 1

            # for the next bb
            i = i + 1

        print ("total #prediction: %d" % i)
        print ("total time: %f" % total_time)
        print ("total positive: %d" % total_positive)
        
        if (i != 0):
            print ("avg prediction speed: %f" % (total_time / i))
            print ("accuracy: %f" % (total_positive / i))

            avg_prediction_speed.append(total_time / i) 
            accuracy.append(total_positive / i)

print(mean(avg_prediction_speed))
print(mean(accuracy))

0 0
video_0317
['0_317_2530b', '0_317_2533b', '0_317_2529b', '0_317_2534b']
total #prediction: 180
total time: 1.725126
total positive: 1
avg prediction speed: 0.009584
accuracy: 0.005556
total #prediction: 83
total time: 0.797792
total positive: 2
avg prediction speed: 0.009612
accuracy: 0.024096
total #prediction: 128
total time: 1.205672
total positive: 5
avg prediction speed: 0.009419
accuracy: 0.039062
total #prediction: 174
total time: 1.654563
total positive: 1
avg prediction speed: 0.009509
accuracy: 0.005747
video_0318
['0_318_2542b']
total #prediction: 164
total time: 1.580843
total positive: 129
avg prediction speed: 0.009639
accuracy: 0.786585
video_0319
['0_319_2548b']
total #prediction: 40
total time: 0.374799
total positive: 0
avg prediction speed: 0.009370
accuracy: 0.000000
video_0320
['0_320_2549b', '0_320_2551b', '0_320_2550b']
total #prediction: 75
total time: 0.714048
total positive: 2
avg prediction speed: 0.009521
accuracy: 0.026667
total #prediction: 83
total ti

total #prediction: 74
total time: 0.711123
total positive: 0
avg prediction speed: 0.009610
accuracy: 0.000000
total #prediction: 53
total time: 0.541804
total positive: 0
avg prediction speed: 0.010223
accuracy: 0.000000
video_0346
['0_346_2739b']
total #prediction: 77
total time: 0.757087
total positive: 2
avg prediction speed: 0.009832
accuracy: 0.025974
0.009625284327485394
0.08257019919395786


In [14]:
avg_prediction_speed = list()
avg_distance = list()
print(len(avg_prediction_speed), len(avg_distance))
for v in videos:
    anno = imdb._get_annotations(v)

    bb_cross={}
    ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

    for i, p in enumerate(ped_b):
        #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
            bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
            #print(p)
    print(v)
    print(ped_b)
    
    for p in ped_b:
      #  print(v, p)
        lstm_seq = ( bb_cross[p])
        #print(type(lstm_seq))
        frames = (anno['ped_annotations'][p]['frames'][0],
        len(anno['ped_annotations'][p]['frames']))

        df = pd.DataFrame(lstm_seq)
        df = df.astype('float32')

        #print(type(scaler))
        #print(scaler.data_max_)
        #print(scaler.data_min_)
        scaler = MinMaxScaler(feature_range=(0, 1))
        lstm_seq = scaler.fit_transform(lstm_seq)
        pred = True
        i = 0
        total_time = 0.0
        total_dist = 0.0

        while( i < lstm_seq.shape[0] - (n_seq + n_seq_future) -1):
            batch = lstm_seq[i:i + n_seq] # n_seq number of element with last index (n_seq-1)

            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            #print(y_hat)
            #print(len(batch))

            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            #print(inv_yhat)

            # future bounding box from gt

            boxA =  df[i+(n_seq + n_seq_future) - 1 :i + (n_seq + n_seq_future)].values[0]
            # future bounding box from prediction
            boxB = inv_yhat

            #print(boxA)
            #print(boxB)

            dist = bb_distance(boxA, boxB)
            total_dist = total_dist + dist
            # for the next bb
            i = i + 1

        print ("total #prediction: %d" % i)
        print ("total time: %f" % total_time)
        
        if (i != 0):
            print ("avg prediction speed: %f" % (total_time / i))
            print ("avg_distance: %f" % (total_dist / i))

            avg_prediction_speed.append(total_time / i) 
            avg_distance.append(total_dist / i)

print(mean(avg_prediction_speed))
print("mean distance", mean(avg_distance))

0 0
video_0317
['0_317_2530b', '0_317_2533b', '0_317_2529b', '0_317_2534b']
844.0 693.5 826.471435546875 703.2152099609375
846.0 696.0 823.314453125 703.7073974609375
847.0 700.0 824.6378173828125 704.5379638671875
849.0 702.5 822.7368774414062 704.6728515625
851.5 705.0 824.0262451171875 704.4025268554688
854.5 707.0 826.5640258789062 704.9659423828125
857.0 709.5 830.4688720703125 705.599853515625
858.0 709.0 843.6629638671875 706.1241455078125
859.5 710.0 846.074951171875 706.5465698242188
860.5 709.5 856.5287475585938 705.3434448242188
862.5 709.5 858.2388916015625 705.3299560546875
863.0 709.0 871.49560546875 702.9984130859375
865.0 709.0 884.3413696289062 701.9210205078125
868.5 707.0 897.077880859375 700.4677734375
870.5 706.0 897.8880615234375 698.697509765625
874.0 704.0 900.7645874023438 697.8128662109375
876.0 705.5 898.6259765625 697.8309326171875
878.0 706.0 884.0524291992188 697.7445068359375
880.5 706.0 868.0660400390625 696.3646240234375
884.0 705.0 856.87646484375 695.

696.5 753.5 695.630859375 739.01171875
699.0 754.5 695.58203125 743.7822875976562
702.0 756.5 697.2656860351562 744.6953125
704.0 757.5 693.52001953125 744.037353515625
711.5 758.0 686.8909912109375 742.6885986328125
718.5 760.0 670.7889404296875 738.6864013671875
726.0 760.5 661.7445068359375 738.4678955078125
733.0 761.5 662.0926513671875 739.4293212890625
739.5 763.0 674.8958740234375 742.5094604492188
746.5 763.5 689.2781982421875 741.2462768554688
754.0 764.5 694.6380615234375 743.7672729492188
761.0 765.0 699.1116333007812 745.827880859375
768.5 766.5 689.0892333984375 746.362060546875
775.5 767.5 678.0875244140625 746.1185302734375
783.0 768.0 666.9364013671875 745.1788940429688
790.0 770.0 659.7978515625 745.2208251953125
797.5 770.5 657.655029296875 744.941650390625
803.5 771.0 657.3515625 744.1542358398438
811.0 773.0 654.611328125 744.1536865234375
818.0 773.5 655.0772705078125 743.8375244140625
825.5 774.5 653.6522827148438 744.1249389648438
832.5 776.0 652.097412109375 744

1076.5 765.5 735.3678588867188 722.2135009765625
1088.5 764.0 736.4632568359375 723.11474609375
1100.5 764.0 736.4146728515625 723.44287109375
1107.5 767.0 736.931640625 725.1915283203125
1116.0 769.5 742.00048828125 728.027099609375
1123.0 772.5 747.5078125 730.7714233398438
1135.5 773.5 755.8935546875 732.7903442382812
1149.5 774.5 767.1338500976562 734.3782348632812
1162.0 775.5 780.4278564453125 736.1582641601562
1188.5 774.0 792.7622680664062 737.543701171875
1214.0 774.0 805.0635986328125 738.2581787109375
1240.5 772.5 812.6979370117188 738.2769775390625
1250.5 775.0 821.4204711914062 737.5579833984375
1259.0 777.0 833.4908447265625 737.6726684570312
1269.0 779.5 842.568603515625 738.1781005859375
1283.0 780.5 851.9910888671875 738.2439575195312
1298.0 783.0 857.369140625 740.5745849609375
1312.0 784.0 861.2026977539062 742.5615234375
1335.0 784.5 865.4888916015625 745.3858642578125
1358.5 786.0 877.0950927734375 748.109375
1381.5 786.5 886.8565673828125 750.771240234375
1405.0 7

1608.0 743.5 1353.321533203125 721.653564453125
1616.0 743.5 1362.059326171875 723.1502685546875
1624.0 743.5 1364.9234619140625 724.56103515625
1632.0 743.5 1373.232421875 725.8009033203125
1641.0 743.5 1386.675537109375 726.7095947265625
1649.0 743.5 1406.11474609375 727.401611328125
1658.5 742.5 1418.42724609375 728.7144775390625
1667.5 741.5 1438.697998046875 731.09326171875
1676.5 741.5 1477.771484375 735.0428466796875
1685.0 741.0 1539.6312255859375 738.9342041015625
1695.0 740.0 1591.2606201171875 740.70654296875
1704.0 740.0 1615.7939453125 742.09716796875
1713.0 739.5 1634.1329345703125 743.0537109375
1723.0 740.5 1647.0521240234375 742.804443359375
1732.0 741.0 1651.5098876953125 740.115966796875
1742.0 742.0 1692.9632568359375 742.0767822265625
1752.0 741.0 1716.9564208984375 745.2268676757812
1762.5 745.0 1686.52197265625 745.8153686523438
1772.5 749.0 1632.4921875 743.2406005859375
1783.5 751.0 1605.9130859375 742.0870361328125
1794.5 752.0 1601.728759765625 738.7058715820

713.5 805.0 829.3615112304688 802.583251953125
710.0 807.0 830.3037109375 805.9256591796875
705.0 808.5 831.8627319335938 807.01318359375
700.5 809.5 829.5623168945312 805.6217041015625
total #prediction: 164
total time: 1.623195
avg prediction speed: 0.009898
avg_distance: 42.634903
video_0319
['0_319_2548b']
682.5 689.0 676.3966674804688 700.1189575195312
684.0 689.0 680.1676635742188 698.6256713867188
686.0 688.5 682.037841796875 695.4168701171875
688.0 689.0 678.890380859375 693.6073608398438
690.0 688.5 679.4444580078125 687.026611328125
692.0 689.0 681.3636474609375 689.0380859375
695.0 689.0 688.519287109375 701.0213623046875
697.0 688.5 692.0279541015625 701.1376953125
700.0 687.0 692.30517578125 699.21240234375
704.0 685.5 688.93115234375 695.0010375976562
707.0 684.5 680.1710205078125 685.636962890625
710.0 684.0 673.9527587890625 692.646240234375
713.0 683.0 672.2992553710938 685.8990478515625
715.5 683.0 675.138671875 689.2763671875
717.5 683.0 673.65087890625 686.197753906

...

568.0 726.0 667.4608154296875 703.6644897460938
564.0 727.0 665.002197265625 704.048095703125
560.0 727.5 658.3045654296875 704.8905029296875
556.0 729.5 651.6551513671875 705.2042236328125
551.5 730.0 649.1644287109375 706.2076416015625
547.5 730.5 651.5797119140625 707.332275390625
542.5 731.5 660.180908203125 706.646240234375
538.0 731.5 663.611328125 706.1371459960938
533.0 732.5 664.8936767578125 704.8753662109375
527.0 731.5 662.6884765625 704.7772216796875
521.5 731.5 658.3651123046875 704.324462890625
515.5 730.5 651.973876953125 704.7991943359375
509.5 729.5 651.2398071289062 705.2967529296875
503.5 728.0 651.10595703125 705.0994873046875
496.0 727.0 651.7711181640625 705.3018798828125
490.0 725.5 646.2447509765625 704.5115356445312
484.0 724.5 639.861328125 704.001953125
478.0 724.5 630.685302734375 704.50390625
472.0 723.5 630.2797241210938 705.6015625
465.0 725.0 632.4453125 707.01220703125
457.5 725.0 633.985107421875 706.760498046875
450.5 726.5 636.3331909179688 705.8038

211.0 773.0 246.29220581054688 761.4449462890625
208.0 774.0 241.60403442382812 760.8015747070312
206.0 775.0 239.28125 760.7982177734375
203.0 776.0 237.8397216796875 761.4938354492188
201.5 777.5 234.03628540039062 761.810546875
199.5 778.5 232.7081298828125 761.9612426757812
197.0 778.5 233.65390014648438 763.1522216796875
196.0 779.5 231.946533203125 763.8712768554688
195.0 781.0 232.28384399414062 763.5128173828125
194.0 781.0 233.995361328125 763.495849609375
195.0 782.5 234.29074096679688 763.0244140625
193.0 781.5 234.83377075195312 762.6619873046875
191.0 780.5 234.02206420898438 762.508056640625
189.0 781.0 234.51419067382812 762.88720703125
187.0 780.5 235.6815185546875 762.8856201171875
186.0 780.5 234.71792602539062 762.9109497070312
184.5 779.0 234.6961669921875 763.2816772460938
183.5 779.0 234.3702392578125 763.95947265625
182.5 779.0 231.89730834960938 764.25634765625
181.5 778.0 232.57763671875 764.899658203125
181.5 778.0 232.43630981445312 765.8419189453125
181.5 77

1080.0 747.0 1002.7822875976562 749.23876953125
1086.0 748.0 1011.6373291015625 749.3399658203125
1091.0 749.0 1015.2061767578125 750.9395751953125
1097.5 751.0 1021.541748046875 751.1011962890625
1103.5 752.0 1024.0517578125 752.6678466796875
1108.5 753.0 1027.52978515625 751.8685302734375
1114.5 754.0 1029.63818359375 751.84130859375
1119.5 756.0 1029.60205078125 751.7550048828125
1124.5 757.0 1034.4906005859375 749.517822265625
1129.0 759.0 1037.563232421875 750.0257568359375
1134.0 760.0 1040.421142578125 751.459716796875
1140.5 759.5 1041.913818359375 750.92919921875
1146.0 759.0 1040.85986328125 754.6410522460938
1152.0 759.5 1041.316650390625 755.4734497070312
1157.5 759.0 1040.89111328125 756.029541015625
1164.0 758.5 1046.490478515625 756.159912109375
1171.0 759.5 1046.16162109375 755.538818359375
1178.5 761.0 1046.57861328125 755.7197265625
1185.5 762.0 1046.187255859375 756.6861572265625
1193.0 763.5 1047.786376953125 758.2765502929688
1200.0 764.5 1046.9151611328125 757.841

782.5 754.0 1047.844482421875 731.48876953125
773.5 751.5 1041.918212890625 730.740234375
765.5 750.0 1037.1866455078125 728.988525390625
757.0 748.5 1032.568603515625 729.289794921875
751.0 747.5 1028.5289306640625 732.521240234375
746.0 746.5 1012.0811767578125 734.91162109375
740.5 745.5 994.07373046875 735.3648681640625
736.0 744.5 978.7356567382812 735.688232421875
729.5 744.0 964.59326171875 736.1340942382812
719.5 740.0 945.462646484375 736.2330322265625
710.0 737.5 921.7330322265625 736.8402709960938
699.0 737.0 907.2200927734375 736.612548828125
688.0 738.5 904.4801025390625 735.2000122070312
676.5 741.5 901.0894775390625 733.62646484375
664.5 746.5 892.8600463867188 734.1590576171875
653.0 752.0 874.690185546875 737.0164794921875
641.0 758.0 855.1522216796875 739.2923583984375
630.0 762.5 828.5908203125 742.3921508789062
619.5 768.5 795.5322875976562 746.0078735351562
609.0 773.5 796.9049072265625 746.045654296875
600.0 779.5 793.2801513671875 747.0270385742188
590.0 785.0 79

213.5 832.0 633.06005859375 776.644287109375
195.0 835.5 640.652099609375 778.794921875
177.5 838.5 639.8294677734375 782.8217163085938
159.0 847.5 616.445068359375 789.4960327148438
142.5 850.0 576.270751953125 791.264404296875
125.0 854.5 530.2003784179688 789.232421875
100.0 853.0 533.445556640625 787.284423828125
81.0 853.0 537.46630859375 787.982177734375
67.5 855.5 523.7620239257812 791.8582763671875
59.0 856.5 507.3929443359375 796.873291015625
total #prediction: 53
total time: 0.514687
avg prediction speed: 0.009711
avg_distance: 317.693979
video_0346
['0_346_2739b']
866.0 738.5 854.570068359375 753.0601806640625
868.0 739.0 854.739013671875 746.5349731445312
868.5 740.0 851.8553466796875 744.587890625
870.5 740.5 861.3431396484375 746.889404296875
872.0 741.0 862.7713012695312 748.0889892578125
874.0 743.0 859.0550537109375 745.7308349609375
875.5 743.5 857.0314331054688 744.5911865234375
878.0 743.0 858.6278076171875 741.0050659179688
880.5 743.0 860.821044921875 738.27185058

In [8]:
#i = 0

#while( i < lstm_seq.shape[0] - (n_seq + n_seq_future)  - 1):
#    print(i)
#    print(lstm_seq[i+59:i+60])
#    print(reframed.iloc[i:i+1, n_seq*4:n_seq*4 + 4]) 
#    i = i + 1
